In [1]:
import numpy as np
import pandas as pd
from io import BytesIO

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from data_preprocessing import get_input_RNN, get_matrix

from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import regularizers

[nltk_data] Downloading package punkt to /Users/peliculas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/peliculas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a sy

In [2]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, filters, kernel_size, dense_units, l2_kernel):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim,
                                trainable = True)(X_input)
    
    X = Conv1D(filters = filters, kernel_size = kernel_size, activation = 'relu',
              kernel_regularizer = regularizers.l2(l2_kernel))(embedding_layer)
    X = MaxPooling1D(pool_size = 2)(X)
    X = Flatten()(X)
    X = Dense(units = dense_units, activation = 'relu')(X)
    X = Dense(units = 1, activation = 'sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [3]:
model = create_model(10000, 500, 100, filters = 32, kernel_size = 8, dense_units = 10, l2_kernel = 0)
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 493, 32)           25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 246, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7872)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                78730     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11 

In [4]:
def grid_search(X, dataset, epochs, models):
    
    Y = dataset.label.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)

    mean_dev_accuracy = np.zeros(np.size(models))
    
    i = 0
    for model in models:
    
        iterations = 5
        dev_accuracy_shuffle_split = np.zeros(iterations)
        shuffle = ShuffleSplit(n_splits = iterations, test_size = 0.2)
        
        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
        w = BytesIO()
        model.save_weights(w) # Save initial weights
        
        j = 0
        for train, dev in shuffle.split(X_train, y_train):
            Xtrain = X_train[train]
            Ytrain = y_train[train]
            Xdev = X_train[dev]
            Ydev = y_train[dev]

            model.fit(Xtrain, Ytrain, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

            loss, accuracy_val = model.evaluate(Xdev, Ydev) # Validate model
            dev_accuracy_shuffle_split[j] = accuracy_val

            model.load_weights(w) # Restore initial weights
            
            j += 1
              
        mean_dev_accuracy[i] = round(np.mean(dev_accuracy_shuffle_split), 3)
        std = round(np.std(dev_accuracy_shuffle_split), 3)
        print('Model ' + str(i) +' --> dev_acc: ' + str(mean_dev_accuracy[i]) + ' +- ' + str(std))
        i += 1
    
    # test
    best_model_index = np.argmax(mean_dev_accuracy)
    best_model = models[best_model_index]
    best_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile best model
    best_model.fit(X_train, y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit best model
    y_pred = best_model.predict(X_test) # Test best model
    y_pred = y_pred > 0.5 # Sigmoid activation function
    accuracy_test = accuracy_score(y_test, y_pred)
    
    return best_model_index, round(accuracy_test, 3)

In [5]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
5,Un paso más cerca de hacer los exámenes 'onlin...,0
6,UNAM REALIZARÁ PRUEBAS ANTIDOPING A ESTUDIANTE...,1
7,Niño de *NUMBER* años se prepara para entrar a...,0
8,*NUMBER* palabras que creíamos inaceptables y ...,0
9,LIMITARÁN EL TIEMPO DE EGRESO EN FILOSOFÍA Y S...,1


#### Models

In [6]:
vocabulary_length = 10000
max_length_sequence = 500
emb_dim = 100
language = 'spanish'
epochs = 7

In [7]:
X, df = get_input_RNN(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(2571, 500)


In [8]:
models = []
filters = [8, 16, 32]
kernel_size = [5, 10]
dense_units = [4, 8, 12]

for i in range(len(filters)):
    for j in range(len(kernel_size)):
        for k in range(len(dense_units)):
            model = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = filters[i], kernel_size = kernel_size[j], dense_units = dense_units[k], l2_kernel = 0)
            models.append(model)

In [9]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6888 - accuracy: 0.5189
Epoch 2/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6345 - accuracy: 0.7603
Epoch 3/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.4427 - accuracy: 0.9075
Epoch 4/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.1507 - accuracy: 0.9751
Epoch 5/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0400 - accuracy: 0.9921
Epoch 6/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0164 - accuracy: 0.9976
Epoch 7/7
412/412 [==============================] - 0s 536us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6906 - accuracy: 0.5140
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6523 - accuracy: 0.5943
Epoch 3/7
1644/1644 [=========

412/412 [==============================] - 0s 458us/step
Model 1 --> dev_acc: 0.675 +- 0.028
Epoch 1/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6858 - accuracy: 0.5255
Epoch 2/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6284 - accuracy: 0.6247
Epoch 3/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.4632 - accuracy: 0.8491
Epoch 4/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.1660 - accuracy: 0.9872
Epoch 5/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0282 - accuracy: 0.9970
Epoch 6/7
1644/1644 [==============================] - 4s 3ms/step - loss: 0.0094 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 0s 686us/step
Epoch 1/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6922 - accuracy: 0.5353
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6610 - accuracy: 0.6101
Epoch 3/7
1644/1644 [===========

1644/1644 [==============================] - 5s 3ms/step - loss: 0.0465 - accuracy: 0.9933
Epoch 5/7
1644/1644 [==============================] - 4s 3ms/step - loss: 0.0153 - accuracy: 0.9970
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0071 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 0s 649us/step
Epoch 1/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6885 - accuracy: 0.5353
Epoch 2/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6090 - accuracy: 0.7597
Epoch 3/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.4073 - accuracy: 0.8929
Epoch 4/7
1644/1644 [==============================] - 6s 4ms/step - loss: 0.1395 - accuracy: 0.9732
Epoch 5/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.0302 - accuracy: 0.9939
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0107 - accuracy: 0.9988
Epoch 7/7
412/412 [===============

1644/1644 [==============================] - 4s 2ms/step - loss: 0.6714 - accuracy: 0.5468
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.4740 - accuracy: 0.7105
Epoch 3/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.1238 - accuracy: 0.9787
Epoch 4/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0152 - accuracy: 0.9970
Epoch 5/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0054 - accuracy: 0.9988
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 7/7
412/412 [==============================] - 0s 447us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6882 - accuracy: 0.5036
Epoch 2/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6568 - accuracy: 0.5918
Epoch 3/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.5421 - accuracy: 0.7281
Epoch 4/7
1644/1644 [=============

1644/1644 [==============================] - 4s 2ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 6/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 7/7
412/412 [==============================] - 0s 354us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6863 - accuracy: 0.5255
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6055 - accuracy: 0.6995
Epoch 3/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.3799 - accuracy: 0.9136
Epoch 4/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0845 - accuracy: 0.9903
Epoch 5/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0107 - accuracy: 0.9982
Epoch 6/7
1644/1644 [==============================] - 4s 3ms/step - loss: 0.0041 - accuracy: 0.9994
Epoch 7/7
412/412 [==============================] - 0s 355us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step -

1644/1644 [==============================] - 6s 3ms/step - loss: 0.6126 - accuracy: 0.6727
Epoch 3/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.3771 - accuracy: 0.8966
Epoch 4/7
1644/1644 [==============================] - 6s 4ms/step - loss: 0.0991 - accuracy: 0.9702
Epoch 5/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.0186 - accuracy: 0.9970
Epoch 6/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.0084 - accuracy: 0.9982
Epoch 7/7
412/412 [==============================] - 0s 568us/step
Epoch 1/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.6932 - accuracy: 0.5152
Epoch 2/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.6760 - accuracy: 0.5657
Epoch 3/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.5952 - accuracy: 0.7360
Epoch 4/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.3722 - accuracy: 0.9088
Epoch 5/7
1644/1644 [=============

1644/1644 [==============================] - 5s 3ms/step - loss: 0.0104 - accuracy: 0.9982
Epoch 7/7
412/412 [==============================] - 0s 728us/step
Epoch 1/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6838 - accuracy: 0.5298
Epoch 2/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6269 - accuracy: 0.6527
Epoch 3/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.4197 - accuracy: 0.8771
Epoch 4/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.1073 - accuracy: 0.9757
Epoch 5/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0143 - accuracy: 0.9988
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0054 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 0s 656us/step
Epoch 1/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.6937 - accuracy: 0.5073
Epoch 2/7
1644/1644 [==============================] - 6s 4ms/step -

1644/1644 [==============================] - 8s 5ms/step - loss: 0.3929 - accuracy: 0.8869
Epoch 4/7
1644/1644 [==============================] - 8s 5ms/step - loss: 0.1024 - accuracy: 0.9641
Epoch 5/7
1644/1644 [==============================] - 6s 4ms/step - loss: 0.0128 - accuracy: 0.9994
Epoch 6/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 7/7
412/412 [==============================] - 0s 786us/step
Epoch 1/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.6857 - accuracy: 0.5152
Epoch 2/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.6651 - accuracy: 0.5541
Epoch 3/7
1644/1644 [==============================] - 6s 4ms/step - loss: 0.6360 - accuracy: 0.5900
Epoch 4/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.5717 - accuracy: 0.6393
Epoch 5/7
1644/1644 [==============================] - 9s 5ms/step - loss: 0.4625 - accuracy: 0.7080
Epoch 6/7
1644/1644 [=============

412/412 [==============================] - 1s 2ms/step
Epoch 1/7
1644/1644 [==============================] - 10s 6ms/step - loss: 0.6896 - accuracy: 0.5030
Epoch 2/7
1644/1644 [==============================] - 11s 6ms/step - loss: 0.6640 - accuracy: 0.5633
Epoch 3/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.5546 - accuracy: 0.7336
Epoch 4/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.3279 - accuracy: 0.8753
Epoch 5/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.0781 - accuracy: 0.9757
Epoch 6/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.0223 - accuracy: 0.9945
Epoch 7/7
412/412 [==============================] - 1s 1ms/step
Model 16 --> dev_acc: 0.714 +- 0.021
Epoch 1/7
1644/1644 [==============================] - 10s 6ms/step - loss: 0.6769 - accuracy: 0.5377
Epoch 2/7
1644/1644 [==============================] - 8s 5ms/step - loss: 0.5836 - accuracy: 0.7068
Epoch 3/7
1644/1644 [===========